### Evaluation

In [1]:
import os
from PIL import Image
import torch
import numpy as np

In [2]:
# Define model path
DATA_PATH = "../../../Data.nosync/"
model_path = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/test649437/checkpoints/iteration_0.pt"

In [3]:
os.chdir('../../encoder4editing/')
from utils.model_utils import setup_model, load_e4e_standalone
from configs import data_configs, paths_config
from utils.common import tensor2im

In [4]:
# Initialize Model
net, opts = setup_model(model_path, device = 'cuda')
encoder = net.encoder
generator = net.decoder
generator.eval()

# Get transforms
dataset_args = data_configs.DATASETS[opts.dataset_type]
transforms_dict = dataset_args['transforms'](opts).get_transforms()

Loading e4e over the pSp framework from checkpoint: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/test649437/checkpoints/iteration_0.pt


In [7]:
def get_inversion_results(image_path):
    # Input
    img_orig = Image.open(image_path).convert('RGB')
    img = transforms_dict['transform_test'](img_orig)
    img = img.reshape(-1, 3, 256, 256)
    img = img.to('cuda').float()

    # Encoder
    latent = encoder(img)

    # Generate
    out_img, _ = generator(latent.unsqueeze(0), input_is_latent=True, randomize_noise=False, return_latents = True)
    reconstruction = tensor2im(out_img[0,:,:,:])

    return reconstruction, latent

In [ ]:
image_path = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/test/1KE21C009-A11.jpg"
get_inversion_results(image_path)